In [27]:
import cv2
from cv2 import aruco
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import os
import matplotlib.pyplot as plt

In [2]:
_pth = os.getcwd()
_pth = os.path.dirname(_pth)
_parent_folder = "data_for_labelling_aug_1_2023"
_calib_folder_name = "calibration_00"


_base_pth = os.path.join(_pth,"recorded_data",_parent_folder)

_webcam_calib_folder = os.path.join(_pth,"recorded_data",_parent_folder,_calib_folder_name)
_webcam_calib_video = os.path.join(_webcam_calib_folder,"webcam_color.msgpack")
_webcam_calib_folder = os.path.join(_webcam_calib_folder)
_webcam_calib_pth = os.path.join( _webcam_calib_folder, "webcam_calibration.msgpack")

In [4]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
detector = aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)
markerLength = 0.05
markerSeperation = 0.01

board = aruco.GridBoard(
        size= [1,1],
        markerLength=markerLength,
        markerSeparation=markerSeperation,
        dictionary=ARUCO_DICT)

In [5]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)
_video_length = 0

for _frame in _video_data:
    _video_length += 1

_video_file.close()

In [6]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

marker_corners = []
object_points = []
first = True
counter = []
rnd = np.random.choice(_video_length, 150, replace=False)
for idx, _frame in enumerate(_video_data):
    
    if idx in rnd:
        _frame = cv2.cvtColor(_frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected_image_points = aruco.detectMarkers(_frame, ARUCO_DICT, parameters=ARUCO_PARAMETERS)
        try:
            if first == True:
                corners_list = corners
                id_list = ids
                first = False
            else:
                corners_list = np.vstack((corners_list, corners))
                id_list = np.vstack((id_list,ids))
            counter.append(len(ids))
        except:
            continue
_video_file.close()

counter = np.array(counter)

In [9]:
calibration_flags = cv2.CALIB_RATIONAL_MODEL
term_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)

In [34]:
mtx2 = np.zeros((3, 3))
dist2 = np.zeros((1, 8))
rvec2 = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(len(counter))]
tvec2 = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(len(counter))]

In [35]:
ret1, mtx1, dist1, rvecs1, tvecs1 = aruco.calibrateCameraAruco(corners_list, id_list, counter, board, _frame.shape[:2], mtx2, dist2, flags=calibration_flags, criteria = term_criteria)

In [41]:
with open(os.path.join(os.path.dirname(_webcam_calib_video), "8p_dist_calib.msgpack"), "wb") as p:
    pckd = mp.packb((mtx1.reshape(1,9), dist1), default=mpn.encode)
    p.write(pckd)
    p.close()